# loading the dataset
the dataset is being loaded from the .csv file which is from https://data.world/datafiniti/hotel-reviews, and we need to retrieve the column with the name reviews.text, and make the ratings as our target with column name review.rating.

In [1]:
!pip install langdetect
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 6.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=0d677b533a336de2c0cad4900bb4a022b1b852d6834a730daf45efb5811403be
  Stored in directory: /Users/bardiadehbasti/Library/Caches/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [4]:
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install tensorflow

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/0f/a5/89a9bdae9f241cad167edb6c890ef2b4c842c5ae81058a2fbfd702dba9bf/tensorflow-2.17.0-cp311-cp311-macosx_12_0_arm64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/a2/ad/e0d3c824784ff121c03cc031f944bc7e139a8f1870ffd2845cc2dd76f6c4/absl_py-2.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for astunparse>=1.6.0 from https://files.pythonhosted.org/packages/2b/03/13dde6512ad7b4557eb792fbcf0c653af6076b81e5941d36ec61f7ce6028/astunparse-1.6.3-py2.py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers>=24.3.25 from https://files.pythonhosted.org/packages/41/f0/7e988a019bc54b2dbd0ad4182ef2d53488bb02e58694cd79d61369e85900/flatbuffers-24.3.25-py2.py3-none-any.whl.metadata
  Obtaining dependency information for gast!=0.5.0,!=0.5.1,!=0.5.2,>=0.2.1 from https://files.pythonhosted.org/packages/a3/61/8001b38

In [35]:
#imports
import pandas as pd
import numpy as np
import re
import langdetect
from langdetect import detect, DetectorFactory
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from nltk.classify import NaiveBayesClassifier
from nltk.classify.util import accuracy as nltk_accuracy
from multiprocessing import Pool

#loading the dataset
reviewsDataset = pd.read_csv('datafiniti-hotel-reviews/7282_1.csv')
reviewsDataset.head()

,address,categories,city,country,latitude,longitude,name,postalCode,province,reviews.date,reviews.dateAdded,reviews.doRecommend,reviews.id,reviews.rating,reviews.text,reviews.title,reviews.userCity,reviews.username,reviews.userProvince
0,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2013-09-22T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,4.0,Pleasant 10 min walk along the sea front to th...,Good location away from the crouds,NaN,Russ (kent),NaN
1,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2015-04-03T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,Really lovely hotel. Stayed on the very top fl...,Great hotel with Jacuzzi bath!,NaN,A Traveler,NaN
2,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2014-05-13T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,Ett mycket bra hotell. Det som drog ner betyge...,Lugnt l��ge,NaN,Maud,NaN
3,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2013-10-27T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,We stayed here for four nights in October. The...,Good location on the Lido.,NaN,Julie,NaN
4,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2015-03-05T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,We stayed here for four nights in October. The...,������ ���������������,NaN,sungchul,NaN


In [36]:
reviewsDataset['reviews.text'].count()

35888

In [37]:
#downliading the nltk library data files for tokenization
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


#initializing lemmatizer and setting English stop words set to variable stop_words
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/bardiadehbasti/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bardiadehbasti/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/bardiadehbasti/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# data cleaning, tokenization
creating a function to check whether the language is indeed english, cheking with RegEx whether the review is symbols and gibberish, and remove any review less than 4 characters.
We also tokenize, and lemmatize the words. which is a NLP technique

In [38]:
#setting seed 0 for langdetect to ensure reproducibility
langdetect.DetectorFactory.seed = 0


#function to check whether the reviews is english or not
def is_english(review):
    try:
        return detect(review) == 'en'
    except langdetect.lang_detect_exception.LangDetectException:
        return False



#function to clean up the reviews, with regex and applying the is_english function, also checking if the review is less than 4 characters
def clean_review(review):
    # Remove non-English reviews
    if not is_english(review):
        return None
    # Remove symbols and gibberish
    if re.match(r'^[^a-zA-Z0-9]+$', review):
        return None
    # Remove very short reviews
    if len(review) < 4:
        return None

    # Tokenize the review
    words = word_tokenize(review)
    # Remove stop words and lemmatize
    cleaned_words = [lemmatizer.lemmatize(word.lower()) for word in words if word.lower() not in stop_words and word.isalpha()]
    
    return ' '.join(cleaned_words)




In [39]:
# Apply cleaning function to reviews.text and keep track of valid indices
valid_indices = []
cleaned_reviews = []

for idx, review in enumerate(reviewsDataset['reviews.text'].fillna('')):
    cleaned_review = clean_review(review)
    if cleaned_review:
        valid_indices.append(idx)
        cleaned_reviews.append(cleaned_review)

In [41]:
# Filter the dataset to keep only valid rows
reviewsDataset = reviewsDataset.iloc[valid_indices].copy()
reviewsDataset['reviews.text'] = cleaned_reviews

# Combine cleaned reviews.text with reviews.title
reviewsDataset['combined_reviews'] = reviewsDataset['reviews.title'].astype(str) + ". " + reviewsDataset['reviews.text']

# Extract combined reviews and ratings
reviews = np.array(reviewsDataset['combined_reviews'])
ratings = np.array(reviewsDataset['reviews.rating'])

# Convert ratings to binary labels (positive: rating >= 3, negative: rating < 3)
ratings = np.array([1 if rating >= 3 else 0 for rating in ratings])

# Verify the length of reviews and ratings
print(f"Number of reviews: {len(reviews)}")
print(f"Number of ratings: {len(ratings)}")

Number of reviews: 33656
Number of ratings: 33656


Result of cleaning: We have eliminated 2232 rows


now we split the data to train, test.

In [42]:
# Split the data into training and testing sets

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(reviews, ratings, test_size=0.25, random_state=42)

In [43]:
#checking out the shape of the train and test set
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(25242,)
(25242,)
(8414,)
(8414,)


Now we prepare features, prepare training data, train Naive Bayes Classifier (it will take 3-4 minutes probably)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense,Flatten, SpatialDropout1D


# Tokenize and pad the text sequences
tokenizer = Tokenizer(num_words=5000, lower=True, split=' ')
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

max_seq_len = 500
X_train = pad_sequences(X_train, maxlen=max_seq_len)
X_test = pad_sequences(X_test, maxlen=max_seq_len)






In [45]:
# Build the LSTM model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=max_seq_len))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

/Users/bardiadehbasti/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [46]:
#now we compile the model and fit the data into the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [47]:
# Train the model (might take more than 10 minutes)
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test), verbose=2)


Epoch 1/5
395/395 - 119s - 302ms/step - accuracy: 0.8599 - loss: 0.3349 - val_accuracy: 0.8842 - val_loss: 0.2832
Epoch 2/5
395/395 - 121s - 308ms/step - accuracy: 0.8976 - loss: 0.2518 - val_accuracy: 0.8847 - val_loss: 0.2640
Epoch 3/5
395/395 - 117s - 297ms/step - accuracy: 0.9115 - loss: 0.2219 - val_accuracy: 0.8853 - val_loss: 0.2668
Epoch 4/5
395/395 - 118s - 298ms/step - accuracy: 0.9193 - loss: 0.2010 - val_accuracy: 0.8855 - val_loss: 0.2850
Epoch 5/5
395/395 - 121s - 307ms/step - accuracy: 0.9276 - loss: 0.1803 - val_accuracy: 0.8823 - val_loss: 0.2868


now we can evaluate the model to get the accuracy, f1-score, precision and recall. 

In [48]:
#evaluate the model
from sklearn.metrics import precision_recall_fscore_support, accuracy_score


# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Test Accuracy: {accuracy:.2f}")


# Predict the sentiment on the test set
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob >= 0.5).astype(int)


# Calculate precision, recall, and f1-score
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='binary')

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

263/263 - 19s - 72ms/step - accuracy: 0.8823 - loss: 0.2868
Test Accuracy: 0.88
263/263 ━━━━━━━━━━━━━━━━━━━━ 19s 73ms/step
Precision: 0.93
Recall: 0.93
F1-Score: 0.93


now we run a sample test to see how it will predict an unseen review

In [49]:
# Function to predict sentiment
def predict_sentiment(review):
    seq = tokenizer.texts_to_sequences([review])
    padded = pad_sequences(seq, maxlen=max_seq_len)
    pred = model.predict(padded)
    return 'positive' if pred >= 0.5 else 'negative'

# Test the prediction function
sample_review = reviewsDataset['combined_reviews'].iloc[0]
print(f"Review: {sample_review}")
print(f"Predicted Sentiment: {predict_sentiment(sample_review)}")
print(f"Actual Rating Sentiment: {'positive' if ratings[0] == 1 else 'negative'}")

Review: Good location away from the crouds. pleasant min walk along sea front water bus restaurant etc hotel comfortable breakfast good quite variety room aircon work well take mosquito repelant
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Predicted Sentiment: positive
Actual Rating Sentiment: positive


What if instead of LSTM based Neural network we used a simple ANN? let's explore the second model 


# using an ANN

In [50]:
#loading the dataset
reviewsDataset = pd.read_csv('datafiniti-hotel-reviews/7282_1.csv')
reviewsDataset.head()

,address,categories,city,country,latitude,longitude,name,postalCode,province,reviews.date,reviews.dateAdded,reviews.doRecommend,reviews.id,reviews.rating,reviews.text,reviews.title,reviews.userCity,reviews.username,reviews.userProvince
0,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2013-09-22T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,4.0,Pleasant 10 min walk along the sea front to th...,Good location away from the crouds,NaN,Russ (kent),NaN
1,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2015-04-03T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,Really lovely hotel. Stayed on the very top fl...,Great hotel with Jacuzzi bath!,NaN,A Traveler,NaN
2,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2014-05-13T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,Ett mycket bra hotell. Det som drog ner betyge...,Lugnt l��ge,NaN,Maud,NaN
3,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2013-10-27T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,We stayed here for four nights in October. The...,Good location on the Lido.,NaN,Julie,NaN
4,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2015-03-05T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,We stayed here for four nights in October. The...,������ ���������������,NaN,sungchul,NaN


In [51]:
# Apply cleaning function to reviews.text and keep track of valid indices
valid_indices = []
cleaned_reviews = []

for idx, review in enumerate(reviewsDataset['reviews.text'].fillna('')):
    cleaned_review = clean_review(review)
    if cleaned_review:
        valid_indices.append(idx)
        cleaned_reviews.append(cleaned_review)

In [52]:
# Filter the dataset to keep only valid rows
reviewsDataset = reviewsDataset.iloc[valid_indices].copy()
reviewsDataset['reviews.text'] = cleaned_reviews

# Combine cleaned reviews.text with reviews.title
reviewsDataset['combined_reviews'] = reviewsDataset['reviews.title'].astype(str) + ". " + reviewsDataset['reviews.text']

# Extract combined reviews and ratings
reviews = np.array(reviewsDataset['combined_reviews'])
ratings = np.array(reviewsDataset['reviews.rating'])

# Convert ratings to binary labels (positive: rating >= 3, negative: rating < 3)
ratings = np.array([1 if rating >= 3 else 0 for rating in ratings])

# Verify the length of reviews and ratings
print(f"Number of reviews: {len(reviews)}")
print(f"Number of ratings: {len(ratings)}")

Number of reviews: 33656
Number of ratings: 33656


In [53]:
# Split the data into training and testing sets

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(reviews, ratings, test_size=0.25, random_state=42)

In [54]:
#checking out the shape of the train and test set
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(25242,)
(25242,)
(8414,)
(8414,)


In [55]:
# Tokenize and pad the text sequences
tokenizer = Tokenizer(num_words=5000, lower=True, split=' ')
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

max_seq_len = 500
X_train = pad_sequences(X_train, maxlen=max_seq_len)
X_test = pad_sequences(X_test, maxlen=max_seq_len)

In [58]:
# Build the ANN model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=max_seq_len))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


/Users/bardiadehbasti/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [59]:
#compiling the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test), verbose=2)

Epoch 1/5
395/395 - 26s - 65ms/step - accuracy: 0.8325 - loss: 0.4658 - val_accuracy: 0.8804 - val_loss: 0.2781
Epoch 2/5
395/395 - 23s - 57ms/step - accuracy: 0.9035 - loss: 0.2332 - val_accuracy: 0.8795 - val_loss: 0.2801
Epoch 3/5
395/395 - 23s - 57ms/step - accuracy: 0.9519 - loss: 0.1328 - val_accuracy: 0.8708 - val_loss: 0.3453
Epoch 4/5
395/395 - 23s - 58ms/step - accuracy: 0.9807 - loss: 0.0631 - val_accuracy: 0.8690 - val_loss: 0.3780
Epoch 5/5
395/395 - 23s - 57ms/step - accuracy: 0.9883 - loss: 0.0409 - val_accuracy: 0.8633 - val_loss: 0.4275


In [60]:
#evaluating the model with accuracy and f1-score, precision, recall

loss, accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Test Accuracy: {accuracy:.2f}")

# Predict the sentiment on the test set
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob >= 0.5).astype(int)

# Calculate precision, recall, and f1-score
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='binary')

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")

263/263 - 2s - 6ms/step - accuracy: 0.8633 - loss: 0.4275
Test Accuracy: 0.86
263/263 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step
Precision: 0.90
Recall: 0.93
F1-Score: 0.92


In [61]:
#testing an unseen review to see how it predicts it
# Function to predict sentiment
def predict_sentiment(review):
    seq = tokenizer.texts_to_sequences([review])
    padded = pad_sequences(seq, maxlen=max_seq_len)
    pred = model.predict(padded)
    return 'positive' if pred >= 0.5 else 'negative'

# Test the prediction function
sample_review = reviewsDataset['combined_reviews'].iloc[0]
print(f"Review: {sample_review}")
print(f"Predicted Sentiment: {predict_sentiment(sample_review)}")
print(f"Actual Rating Sentiment: {'positive' if ratings[0] == 1 else 'negative'}")

Review: Good location away from the crouds. pleasant min walk along sea front water bus restaurant etc hotel comfortable breakfast good quite variety room aircon work well take mosquito repelant
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Predicted Sentiment: positive
Actual Rating Sentiment: positive


# using RoBERTa

In [62]:
!pip install transformers

In [71]:
!pip install torch

  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/d0/5f/f41b14a398d484bf218d5167ec9061c1e76f500d9e25166117818c8bacda/torch-2.3.1-cp311-none-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for typing-extensions>=4.8.0 from https://files.pythonhosted.org/packages/26/9f/ad63fc0248c5379346306f8668cda6e2e2e9c95e01216d2b8ffd9ff037d0/typing_extensions-4.12.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 MB 26.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1


In [76]:
!pip install tensorflow

In [83]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
import torch

In [65]:
#redoing all the previous steps to load dataset, clean the data, with a different tokenization method, (using AutoTokenize which is from the transformers library)

#loading the dataset
reviewsDataset = pd.read_csv('datafiniti-hotel-reviews/7282_1.csv')
reviewsDataset.head()

,address,categories,city,country,latitude,longitude,name,postalCode,province,reviews.date,reviews.dateAdded,reviews.doRecommend,reviews.id,reviews.rating,reviews.text,reviews.title,reviews.userCity,reviews.username,reviews.userProvince
0,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2013-09-22T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,4.0,Pleasant 10 min walk along the sea front to th...,Good location away from the crouds,NaN,Russ (kent),NaN
1,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2015-04-03T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,Really lovely hotel. Stayed on the very top fl...,Great hotel with Jacuzzi bath!,NaN,A Traveler,NaN
2,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2014-05-13T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,Ett mycket bra hotell. Det som drog ner betyge...,Lugnt l��ge,NaN,Maud,NaN
3,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2013-10-27T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,We stayed here for four nights in October. The...,Good location on the Lido.,NaN,Julie,NaN
4,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2015-03-05T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,We stayed here for four nights in October. The...,������ ���������������,NaN,sungchul,NaN


In [66]:
# Apply cleaning function to reviews.text and keep track of valid indices
valid_indices = []
cleaned_reviews = []

for idx, review in enumerate(reviewsDataset['reviews.text'].fillna('')):
    cleaned_review = clean_review(review)
    if cleaned_review:
        valid_indices.append(idx)
        cleaned_reviews.append(cleaned_review)

In [67]:
# Filter the dataset to keep only valid rows
reviewsDataset = reviewsDataset.iloc[valid_indices].copy()
reviewsDataset['reviews.text'] = cleaned_reviews

# Combine cleaned reviews.text with reviews.title
reviewsDataset['combined_reviews'] = reviewsDataset['reviews.title'].astype(str) + ". " + reviewsDataset['reviews.text']

# Extract combined reviews and ratings
reviews = np.array(reviewsDataset['combined_reviews'])
ratings = np.array(reviewsDataset['reviews.rating'])

# Convert ratings to binary labels (positive: rating >= 3, negative: rating < 3)
ratings = np.array([1 if rating >= 3 else 0 for rating in ratings])

# Verify the length of reviews and ratings
print(f"Number of reviews: {len(reviews)}")
print(f"Number of ratings: {len(ratings)}")

Number of reviews: 33656
Number of ratings: 33656


In [68]:
X_train, X_test, y_train, y_test = train_test_split(reviews, ratings, test_size=0.25, random_state=42)

In [69]:
#checking out the shape of the train and test set
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(25242,)
(25242,)
(8414,)
(8414,)


now we load the RoBERTa model from hugging face, and the tokenizer from AutoTokenizer. 
here is the link to the model used:
https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest

In [85]:
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)


ImportError: 
AutoModelForSequenceClassification requires the PyTorch library but it was not found in your environment.
However, we were able to find a TensorFlow installation. TensorFlow classes begin
with "TF", but are otherwise identically named to our PyTorch classes. This
means that the TF equivalent of the class you tried to import would be "TFAutoModelForSequenceClassification".
If you want to use TensorFlow, please use TF classes instead!

If you really do want to use PyTorch please go to
https://pytorch.org/get-started/locally/ and follow the instructions that
match your environment.
